In [1]:
%store -r params
%store -r secrets

In [2]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.avro.functions import to_avro, from_avro
from pyspark.sql.functions import col, struct, lit, unbase64
from pyspark.sql.types import IntegerType
import random

In [3]:
spark = SparkSession.builder.appName("Tokenization").getOrCreate()

24/03/31 13:22:58 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
with open(params["avro_schema_file"]) as f:
    avsc = f.read()

In [6]:
read_checkpoint_location = params["checkpoints"]["tokenization_read"]
os.makedirs(read_checkpoint_location, exist_ok=True)
write_checkpoint_location = params["checkpoints"]["tokenization_write"]
os.makedirs(write_checkpoint_location, exist_ok=True)

In [7]:
raw_avro_df = (spark.readStream
          .format("kafka")
          .option("kafka.bootstrap.servers", secrets["RED_KAFKA_SERVERS"])
          .option("kafka.sasl.mechanism", "PLAIN")
          .option("kafka.security.protocol", "SASL_SSL")
          .option("kafka.sasl.jaas.config", "org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(secrets["RED_KAFKA_USERNAME"], secrets["RED_KAFKA_PASSWORD"]))
          .option("subscribe", params["topic"]["red_landing"])
          .option("kafka.group.id", "tokenization")
          .option('checkpointLocation', read_checkpoint_location)
          .load()
        )

24/03/31 13:23:42 WARN KafkaSourceProvider: Kafka option 'kafka.group.id' has been set on this query, it is
 not recommended to set this option. This option is unsafe to use since multiple concurrent
 queries or sources using the same group id will interfere with each other as they are part
 of the same consumer group. Restarted queries may also suffer interference from the
 previous run having the same group id. The user should have only one query per group id,
 and/or set the option 'kafka.session.timeout.ms' to be very small so that the Kafka
 consumers from the previous query are marked dead by the Kafka group coordinator before the
 restarted query starts running.
    


In [8]:
raw_json_df = raw_avro_df.select("key","value").withColumn("value", from_avro("value", avsc))
tokenized_json_df = raw_json_df.withColumn("value", col("value").withField("cc_num",lit(random.randrange(1000000000000000,9999999999999999))))
tokenized_avro_df = tokenized_json_df.withColumn("value",to_avro(col("value"), avsc))

In [10]:
sQuery = (tokenized_avro_df
        .writeStream
        .format("kafka")
        .queryName("tokenization")
        .option("kafka.bootstrap.servers", secrets["GREEN_KAFKA_SERVERS"])
        .option("kafka.sasl.mechanism", "PLAIN")
        .option("kafka.security.protocol", "SASL_SSL")
        .option("kafka.sasl.jaas.config", "org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(secrets["GREEN_KAFKA_USERNAME"], secrets["GREEN_KAFKA_PASSWORD"]))
        .option("topic",params["topic"]["green_landing"])
        .option("checkpointLocation",write_checkpoint_location)
        .outputMode("append")
        .start()
        )

24/03/31 13:25:08 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/03/31 13:25:08 WARN KafkaSourceProvider: Kafka option 'kafka.group.id' has been set on this query, it is
 not recommended to set this option. This option is unsafe to use since multiple concurrent
 queries or sources using the same group id will interfere with each other as they are part
 of the same consumer group. Restarted queries may also suffer interference from the
 previous run having the same group id. The user should have only one query per group id,
 and/or set the option 'kafka.session.timeout.ms' to be very small so that the Kafka
 consumers from the previous query are marked dead by the Kafka group coordinator before the
 restarted query starts running.
    
24/03/31 13:25:08 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, group.id, auto.offset.reset]' we